# From YOLO.ipynb

## Obtaining baseline results using Yolo models 

### Data

#### Converting Geojson to YOLO

In [ ]:
import os
import numpy as np
from glob import glob
from PIL import Image

def fix_path(path: str) -> str:
    return path.replace("\\", "/")

# Directories
labels_dir = fix_path(r"datasets\Planet\labels")
source_dir = fix_path(r"datasets\Planet\jpg_source")
output_dir = fix_path("datasets/Planet/yolo_labels")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)


In [ ]:
import os
import numpy as np
from glob import glob
from PIL import Image
from tqdm import tqdm

# Class mapping
class_map = {
    "marine_debris": 0  # Update with additional classes if needed
}

# Normalize bounding box coordinates
def normalize_bbox(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

# Process .npy files
npy_files = glob(os.path.join(labels_dir, "*.npy"))

for npy_file in tqdm(npy_files):
    # Load .npy data
    data = np.load(npy_file, allow_pickle=True).tolist()

    # Get corresponding image dimensions
    base_name = os.path.splitext(os.path.basename(npy_file))[0]
    image_path = os.path.join(source_dir, base_name + ".jpg")
    
    if not os.path.exists(image_path):
        print(f"Image not found for {base_name}, skipping...")
        continue

    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Create YOLO label file
    yolo_file_path = os.path.join(output_dir, base_name + ".txt")
    with open(yolo_file_path, "w") as yolo_file:
        for entry in data:
            x_min, y_min, x_max, y_max, class_id = entry
            if class_id > 0:
                class_id -= 1

            # Normalize the bounding box coordinates
            x_center, y_center, width, height = normalize_bbox(
                (x_min, y_min, x_max, y_max), img_width, img_height
            )

            # Write to YOLO file
            yolo_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print("Conversion complete!")


  0%|          | 0/739 [00:00<?, ?it/s]

100%|██████████| 739/739 [00:08<00:00, 86.18it/s]

Conversion complete!


### Splitting Dataset

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
labels_dir = "datasets/Planet/yolo_labels"
images_dir = "datasets/Planet/jpg_source"
output_dir = "datasets/Planet/dataset_splits"

# Ratios
train_ratio = 0.75
val_ratio = 0.15
test_ratio = 0.1

# Ensure the ratios sum to 1
assert round(train_ratio + val_ratio + test_ratio, 5) == 1.0, "Ratios must sum to 1."

# Get list of label files
label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]

# Split into train, validation, and test
train_files, temp_files = train_test_split(label_files, test_size=(1 - train_ratio), random_state=42, shuffle=True)
val_files, test_files = train_test_split(temp_files, test_size=(test_ratio / (test_ratio + val_ratio)), random_state=42, shuffle=True)

# Function to move files
def move_files(files, dest_images, dest_labels):
    os.makedirs(dest_images, exist_ok=True)
    os.makedirs(dest_labels, exist_ok=True)
    for label_file in files:
        # Move label file
        src_label = os.path.join(labels_dir, label_file)
        dest_label = os.path.join(dest_labels, label_file)
        shutil.copy(src_label, dest_label)

        # Move corresponding image file
        image_file = label_file.replace(".txt", ".jpg")  # Assuming images are .jpg
        src_image = os.path.join(images_dir, image_file)
        dest_image = os.path.join(dest_images, image_file)
        if os.path.exists(src_image):
            shutil.copy(src_image, dest_image)
        else:
            print(f"Image file {image_file} not found for label {label_file}")

# Move files to respective directories
move_files(train_files, os.path.join(output_dir, "train/images"), os.path.join(output_dir, "train/labels"))
move_files(val_files, os.path.join(output_dir, "val/images"), os.path.join(output_dir, "val/labels"))
move_files(test_files, os.path.join(output_dir, "test/images"), os.path.join(output_dir, "test/labels"))

print("Dataset split completed.")
print(f"Train files: {len(train_files)}")
print(f"Validation files: {len(val_files)}")
print(f"Test files: {len(test_files)}")


Dataset split completed.
Train files: 504
Validation files: 101
Test files: 68


#### Augmentation

In [ ]:
import os
import shutil
import random
from glob import glob
from PIL import Image, ImageOps

def fix_path(path: str) -> str:
    return path.replace("\\", "/")

# Directories
labels_dir = fix_path(r"yolo_labels")
source_dir = fix_path(r"NASA_Planet_Data\source")
output_dir = fix_path(r"aug")

# Ensure output directories exist
os.makedirs(output_dir, exist_ok=True)
augmented_images_dir = os.path.join(output_dir, "images")
augmented_labels_dir = os.path.join(output_dir, "labels")
os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(augmented_labels_dir, exist_ok=True)

# Augmentation functions
def augment_image(image, label_path, base_name, augmentation_type):
    augmented_image = image.copy()
    if augmentation_type == "flip":
        augmented_image = ImageOps.mirror(image)
    elif augmentation_type == "rotate":
        augmented_image = image.rotate(90, expand=True)
    
    # Save augmented image
    augmented_image_path = os.path.join(augmented_images_dir, f"{base_name}_{augmentation_type}.jpg")
    augmented_image.save(augmented_image_path)

    # Copy corresponding label file
    if os.path.exists(label_path):
        augmented_label_path = os.path.join(augmented_labels_dir, f"{base_name}_{augmentation_type}.txt")
        shutil.copy(label_path, augmented_label_path)
    else:
        print(f"Label file {label_path} missing for image {base_name}, skipping...")

# Process images and labels
image_files = glob(os.path.join(source_dir, "*.jpg"))
for image_path in image_files:
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    label_path = os.path.join(labels_dir, f"{base_name}.txt")

    if not os.path.exists(label_path):
        print(f"Label file missing for {base_name}, skipping...")
        continue

    with Image.open(image_path) as img:
        # Save original image and label
        img.save(os.path.join(augmented_images_dir, f"{base_name}.jpg"))
        shutil.copy(label_path, os.path.join(augmented_labels_dir, f"{base_name}.txt"))

        # Apply augmentations
        augment_image(img, label_path, base_name, "flip")
        augment_image(img, label_path, base_name, "rotate")

print("Dataset augmentation completed.")


Dataset augmentation completed.


In [ ]:
import os

# Directories for augmented dataset
augmented_images_dir = r"aug\images"
augmented_labels_dir = r"aug\labels"

# Count the number of image and label files
num_images = len([f for f in os.listdir(augmented_images_dir) if f.endswith(".jpg")])
num_labels = len([f for f in os.listdir(augmented_labels_dir) if f.endswith(".txt")])

# Print the counts
print(f"Number of augmented images: {num_images}")
print(f"Number of augmented labels: {num_labels}")

# Ensure the counts match
if num_images != num_labels:
    print("Warning: The number of images and labels do not match!")
else:
    print("Dataset is consistent: Images and labels match.")


Number of augmented images: 2217
Number of augmented labels: 2217
Dataset is consistent: Images and labels match.


#### Prepare val and test sets for augmentation and fair validation runs

---

# Main Training

In [1]:
# Check pytorch gpu detection
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
1
NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
import os
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
project = "runs/train"
name = "yolo11n_grey"
os.makedirs(project, exist_ok=True)

In [4]:
# Train the model
results = model.train(
    data="datasets/Planet/dataset_splits_aug_grey/data.yaml",
    epochs=100,
    imgsz=256,
    batch=16,
    device='cuda',
    optimizer="SGD",
    workers=8,
    lr0=0.001,
    patience=20,
    project=project,
    name=name,
    plots=True,
)

New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.73  Python-3.10.16 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=datasets/Planet/dataset_splits_aug_grey/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=runs/train, name=yolo11n_grey2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, ret

train: Scanning C:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug_grey\train\labels.cache... 2868 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2868/2868 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug_grey\val\labels.cache... 574 images, 0 backgrounds, 0 corrupt: 100%|██████████| 574/574 [00:00<?, ?it/s]


Plotting labels to runs\train\yolo11n_grey2\labels.jpg... 
optimizer: SGD(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs\train\yolo11n_grey2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        574       1978      0.696      0.648      0.666      0.372

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     55/100     0.503G      1.423      1.156      1.064         14        256: 100%|██████████| 180/180 [00:20<00:00,  8.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:02<00:00,  6.18it/s]


                   all        574       1978      0.721       0.64      0.664      0.372
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 35, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

55 epochs completed in 0.358 hours.
Optimizer stripped from runs\train\yolo11n_grey2\weights\last.pt, 5.4MB
Optimizer stripped from runs\train\yolo11n_grey2\weights\best.pt, 5.4MB

Validating runs\train\yolo11n_grey2\weights\best.pt...
Ultralytics 8.3.73  Python-3.10.16 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:03<00:00,  5.07it/s]


                   all        574       1978      0.693      0.674      0.703      0.394
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\train\yolo11n_grey2


In [ ]:
import os
from glob import glob
from ultralytics import YOLO

images = list(sorted(glob("datasets/Planet/dataset_splits_aug/val/images/*.jpg")))

# Load a model
model = YOLO("runs/train/train3/weights/best.pt")  # pretrained YOLO11 model

# Run batched inference on a list of images
results = model(
    images,
    iou=0.5,
    imgsz=256,
    device='cuda',
    project="runs/detect",
    name="val_v11s",
    save=True,
    save_txt=True,
    show_labels=True,
    show_conf=True,
    save_conf=True
)  # return a list of Results objects



0: 256x256 1 debris, 2.1ms
1: 256x256 3 debriss, 2.1ms
2: 256x256 (no detections), 2.1ms
3: 256x256 (no detections), 2.1ms
4: 256x256 1 debris, 2.1ms
5: 256x256 (no detections), 2.1ms
6: 256x256 (no detections), 2.1ms
7: 256x256 (no detections), 2.1ms
8: 256x256 1 debris, 2.1ms
9: 256x256 1 debris, 2.1ms
10: 256x256 1 debris, 2.1ms
11: 256x256 (no detections), 2.1ms
12: 256x256 3 debriss, 2.1ms
13: 256x256 2 debriss, 2.1ms
14: 256x256 1 debris, 2.1ms
15: 256x256 2 debriss, 2.1ms
16: 256x256 1 debris, 2.1ms
17: 256x256 (no detections), 2.1ms
18: 256x256 (no detections), 2.1ms
19: 256x256 2 debriss, 2.1ms
20: 256x256 2 debriss, 2.1ms
21: 256x256 2 debriss, 2.1ms
22: 256x256 1 debris, 2.1ms
23: 256x256 1 debris, 2.1ms
24: 256x256 1 debris, 2.1ms
25: 256x256 (no detections), 2.1ms
26: 256x256 3 debriss, 2.1ms
27: 256x256 (no detections), 2.1ms
28: 256x256 1 debris, 2.1ms
29: 256x256 (no detections), 2.1ms
30: 256x256 3 debriss, 2.1ms
31: 256x256 5 debriss, 2.1ms
32: 256x256 5 debriss, 2.1

#### BBox post processing (box merging)

In [ ]:
import os
from glob import glob
from ultralytics import YOLO

images = list(sorted(glob("datasets/Planet/dataset_splits_aug/val/images/*.jpg")))

# Load a model
model = YOLO("runs/train/train3/weights/best.pt")  # pretrained YOLO11 model
project = "runs/detect"  # save results to project folder
name = "post_process_this"

# Run batched inference on a list of images
results = model(
    images,
    iou=0.5,
    imgsz=256,
    device='cuda',
    project=project,
    name=name,
    save=True,
    save_txt=True,
    show_labels=True,
    show_conf=True,
    save_conf=True
)  # return a list of Results objects



0: 256x256 2 debriss, 0.6ms
1: 256x256 1 debris, 0.6ms
2: 256x256 1 debris, 0.6ms
3: 256x256 1 debris, 0.6ms
4: 256x256 1 debris, 0.6ms
5: 256x256 2 debriss, 0.6ms
6: 256x256 4 debriss, 0.6ms
7: 256x256 3 debriss, 0.6ms
8: 256x256 4 debriss, 0.6ms
9: 256x256 3 debriss, 0.6ms
10: 256x256 3 debriss, 0.6ms
11: 256x256 3 debriss, 0.6ms
12: 256x256 1 debris, 0.6ms
13: 256x256 (no detections), 0.6ms
14: 256x256 (no detections), 0.6ms
15: 256x256 (no detections), 0.6ms
16: 256x256 1 debris, 0.6ms
17: 256x256 (no detections), 0.6ms
18: 256x256 1 debris, 0.6ms
19: 256x256 1 debris, 0.6ms
20: 256x256 1 debris, 0.6ms
21: 256x256 1 debris, 0.6ms
22: 256x256 1 debris, 0.6ms
23: 256x256 1 debris, 0.6ms
24: 256x256 1 debris, 0.6ms
25: 256x256 1 debris, 0.6ms
26: 256x256 1 debris, 0.6ms
27: 256x256 1 debris, 0.6ms
28: 256x256 1 debris, 0.6ms
29: 256x256 1 debris, 0.6ms
30: 256x256 (no detections), 0.6ms
31: 256x256 (no detections), 0.6ms
32: 256x256 (no detections), 0.6ms
33: 256x256 (no detections),

In [7]:
import os
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
from app import increment_path, process_tif, merge_overlapping_boxes
from PIL import ImageDraw, Image

IMAGE_DIMENSIONS = (256, 256)
IMAGE_AREA = IMAGE_DIMENSIONS[0] * IMAGE_DIMENSIONS[1]

model_path = "runs/train/train3/weights/best.pt"
model = YOLO(model_path)

# Using PLANETSCOPE model
inference_dir = r"runs/detect/inference_merged"
inference_dir = increment_path(inference_dir)
os.makedirs(inference_dir, exist_ok=True)


def detect_marine_debris(image_path: str):
    # Run inference
    results = model(image_path, iou=0.8)
    result = results[0]  # Assume a single result

    # Open the original image
    bounds = None
    if image_path.lower().endswith(".jpg"):
        img = Image.open(image_path).convert("RGB")
        draw = ImageDraw.Draw(img)
    else:
        img, bounds = process_tif(image_path)
        if img is None:  # Handle failed TIFF processing
            return None, None  # Stop processing if TIFF conversion failed
        draw = ImageDraw.Draw(img)
        longitude_width = bounds.right - bounds.left
        latitude_height = bounds.top - bounds.bottom

    # Draw bounding boxes
    bboxes = []
    for box in result.boxes.xywh.tolist():
        x_center, y_center, width, height = box
        x0 = int(x_center - width / 2)
        y0 = int(y_center - height / 2)
        x1 = int(x_center + width / 2)
        y1 = int(y_center + height / 2)

        bboxes.append([x0, y0, x1, y1])

    # Merge overlapping boxes
    merged_boxes = merge_overlapping_boxes(bboxes)
    # merged_boxes = bboxes

    # Draw merged bounding boxes
    latlong_boxes = []
    for box in merged_boxes:
        x0, y0, x1, y1 = box
        if bounds:
            x0_latlong = ((x0 / img.width) * longitude_width + bounds.left)
            y0_latlong = ((y0 / img.height) * latitude_height + bounds.bottom)
            x1_latlong = ((x1 / img.width) * longitude_width + bounds.left)
            y1_latlong = ((y1 / img.height) * latitude_height + bounds.bottom)
            latlong_boxes.append([x0_latlong, y0_latlong, x1_latlong, y1_latlong])
        draw.rectangle(box, outline="red", width=3)

    # Save the image
    img.save(os.path.join(inference_dir, os.path.basename(image_path)))  # or .png, .tif, etc. 


images = list(sorted(glob("datasets/Planet/dataset_splits_aug/val/images/*.jpg")))
for image in tqdm(images):
    detect_marine_debris(image)

  0%|          | 0/574 [00:00<?, ?it/s]


image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20160928_153233_0e16_16816-29825-16_jpg.rf.c6878d46d3fa246b66ec24e637a35b3d.jpg: 256x256 2 debriss, 12.1ms
Speed: 1.0ms preprocess, 12.1ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)


  0%|          | 1/574 [00:00<01:19,  7.20it/s]


image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20160928_153233_0e16_16816-29825-16_jpg_0.000000.jpg: 256x256 1 debris, 12.3ms
Speed: 1.0ms preprocess, 12.3ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20160928_153233_0e16_16816-29825-16_jpg_1.000000.jpg: 256x256 1 debris, 13.0ms
Speed: 0.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20160928_153233_0e16_16816-29825-16_jpg_2.000000.jpg: 256x256 1 debris, 12.8ms
Speed: 0.0ms preprocess, 12.8ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20160928_153233_0e16_16816-29825-16_jpg_3.000000.jpg: 256x256 1 debris, 12.0ms
Speed: 1.0ms 

  1%|          | 6/574 [00:00<00:21, 25.92it/s]

...


image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20181031_095925_103b_32710-31766-16_jpg.rf.a79a3024b70098d821e06db6d8aef793.jpg: 256x256 4 debriss, 22.5ms
Speed: 1.0ms preprocess, 22.5ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20181031_095925_103b_32710-31766-16_jpg_0.000000.jpg: 256x256 2 debriss, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20181031_095925_103b_32710-31766-16_jpg_1.000000.jpg: 256x256 2 debriss, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20181031_095925_103b_32710-31766-16_jpg_2.000000.jpg: 256x256 

 86%|████████▌ | 495/574 [00:11<00:01, 41.27it/s]


image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20181031_095925_103b_32710-31766-16_jpg_4.000000.jpg: 256x256 2 debriss, 21.0ms
Speed: 0.0ms preprocess, 21.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20181031_095925_103b_32727-31766-16_jpg.rf.9f51422218bb4bfbc58219d4f5187afc.jpg: 256x256 9 debriss, 11.2ms
Speed: 1.0ms preprocess, 11.2ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20181031_095925_103b_32727-31766-16_jpg_0.000000.jpg: 256x256 10 debriss, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

image 1/1 c:\Users\hifia\Projects\Marine_Debris\datasets\Planet\dataset_splits_aug\val\images\20181031_095925_103b_32727-31766-16_jpg_1.000000.jpg: 256x256

 87%|████████▋ | 499/574 [01:08<00:10,  7.26it/s]


KeyboardInterrupt: 